In [36]:
import requests
from bs4 import BeautifulSoup

In [37]:
url='https://www.imdb.com/chart/top/'

In [43]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
response=requests.get(url,headers=headers)
content=response.content

In [44]:
soup=BeautifulSoup(content,'html.parser')

In [59]:
#hyper_l=soup.find_all('class':'ipc-title-link-wrapper','a')
hyper_l=soup.find_all('a', class_='ipc-title-link-wrapper')

In [68]:
href_list=[a.get('href') for a in hyper_l if str(a.get('href')).split('/')[1]!='chart' ] 

In [77]:
i=0
for q in range(0,9):
    for w in range(0,9):
        for e in range(0,9):
            for r in range(0,9):
                for t in range(0,9):
                    for y in range(0,9):
                        for u in range(1,9):
                            w_url='https://www.imdb.com/title/tt'+str(q)+str(w)+str(e)+str(r)+str(t)+str(y)+str(u)+'/'
                            response=requests.get(url,headers=headers)
                            content=response.content
                            soup=BeautifulSoup(content)
                            title=
                            
                            i=i+1
                            print(i,w_url)
                            break
                        break
                    break
                break
            break
        break
    break


                        

1 https://www.imdb.com/title/tt0000001/
